In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import os
import random
from PIL import Image

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [23]:
img_shape = np.array([28,28])
num_input = img_shape[0] * img_shape[1]
num_classes = 10

drop_out_prob = 0.5
learning_rate = 0.001
num_steps = 100
batch_size = 128
display_step = 20

In [24]:
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
drop_out = tf.placeholder(tf.float32)

In [32]:
def conv_net(x, drop_out):
    x = tf.reshape(x, shape=[-1, img_shape[0], img_shape[1], 1])
    
    conv1 = tf.layers.conv2d(
        inputs=x,
        filters=32,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)
    
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5,5],
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=2)
    
    pool2_flat = tf.reshape(pool2,[-1,7*7*64])
    dense = tf.layers.dense(inputs=pool2_flat,units=1024,activation=tf.nn.relu)
    dropout = tf.layers.dropout(
        inputs=dense,
        rate=drop_out)

    return tf.layers.dense(inputs=dropout,units=num_classes)

In [33]:
logits = conv_net(X,drop_out)
prediction = tf.nn.softmax(logits)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(prediction), reduction_indices=[1]))

correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [34]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    test_images,test_labels = mnist.test.next_batch(batch_size)
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(train_step, feed_dict={X: batch_x, Y: batch_y,drop_out:drop_out_prob})
        if step % display_step == 0 or step == 1:
            print("step:",step)
            pred = sess.run(prediction,feed_dict={X: test_images})
            print("prediction:",np.argmax(pred,axis=1)[:5])
            print("y predicts:",np.argmax(test_labels,axis=1)[:5])
            print("Cost:",sess.run(cross_entropy,feed_dict={X:test_images,Y:test_labels,drop_out:0}))
            print("Accuracy:",sess.run(accuracy, feed_dict={X:test_images,Y:test_labels,drop_out:0}))
            print("________________________________________")

    print("Accuracy:")
    print(num_steps, sess.run(accuracy, feed_dict={X:test_images, Y:test_labels,drop_out:0}))
            
    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:",
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels,
                                      drop_out:0}))

step: 1
prediction: [6 6 0 7 0]
y predicts: [1 2 0 7 5]
Cost: 2.184853
Accuracy: 0.265625
________________________________________
step: 20
prediction: [1 2 0 7 5]
y predicts: [1 2 0 7 5]
Cost: 0.56226885
Accuracy: 0.8359375
________________________________________
step: 40
prediction: [1 8 0 7 5]
y predicts: [1 2 0 7 5]
Cost: 0.2568093
Accuracy: 0.890625
________________________________________
step: 60
prediction: [1 2 0 7 5]
y predicts: [1 2 0 7 5]
Cost: 0.11632003
Accuracy: 0.96875
________________________________________
step: 80
prediction: [1 2 0 7 5]
y predicts: [1 2 0 7 5]
Cost: 0.07647233
Accuracy: 0.984375
________________________________________
step: 100
prediction: [1 2 0 7 5]
y predicts: [1 2 0 7 5]
Cost: 0.065568626
Accuracy: 0.984375
________________________________________
Accuracy:
100 0.984375
Optimization Finished!
Testing Accuracy: 0.9722
